# Extração de Características

Load

In [5]:
import numpy as np

y = np.array([0,1,2,3,4,5,6,7] * 99)

voluntarios = dict()
for v in range(10):
    voluntarios[v] = np.load(f'./datasets/caract_{v}.npy')

(792,)
{0: (792, 40), 1: (792, 40), 2: (792, 40), 3: (792, 40), 4: (792, 40), 5: (792, 40), 6: (792, 40), 7: (792, 40), 8: (792, 40), 9: (792, 40)}
